In [1]:
#Importamos la librería pandas que necesitamos para la lectura, conversión y limpieza de los datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np
import re 

#Importamos librerías necesarias para la visualización
# -----------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns

# Imputación de nulos usando métodos avanzados estadísticos
# -----------------------------------------------------------------------
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer

pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames
pd.set_option('display.max_rows', None) # para poder visualizar todas las filas de los DataFrames

In [2]:
# Importar librería para la conexión con MySQL
# -----------------------------------------------------------------------
import mysql.connector
from mysql.connector import errorcode



In [3]:
df_HR = pd.read_csv('df_HR_limpio.csv')
df_HR.head()

,Age,Attrition,BusinessTravel,DailyRate,DistanceFromHome,Education,EducationField,EmployeeNumber,EnvironmentSatisfaction,Gender,HourlyRate,JobInvolvement,JobLevel,JobRole,JobSatisfaction,MaritalStatus,MonthlyIncome,MonthlyRate,NumCompaniesWorked,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager,DateBirth,RemoteWork
0,51,No,Unknown,684.0,6,3,Unknown,"162,0",1,M,51.0,3,5,Research Director,3,Unknown,19537.0,6462,7,No,13,"3,0",3,0,Unknown,5,3.000000,20,15,15,1972,yes
1,52,No,Unknown,699.0,1,4,Life Sciences,"259,0",3,M,65.0,2,5,Manager,3,Unknown,19999.0,5678,0,Unknown,14,"3,0",1,1,"34,0",5,3.000000,33,11,9,1971,yes
2,42,No,travel_rarely,532.0,4,2,Technical Degree,"319,0",3,M,58.0,3,5,Manager,4,Married,19232.0,4933,1,No,11,"3,0",4,0,"22,0",3,2.766568,22,11,15,1981,yes
3,47,No,travel_rarely,359.0,2,4,Medical,Unknown,1,F,82.0,3,4,Research Director,3,Married,17169.0,26703,3,Unknown,19,"3,0",2,2,Unknown,2,2.766568,20,5,6,1976,no
4,48,No,Unknown,117.0,22,3,Medical,"1900,0",4,F,58.0,3,4,Manager,4,Unknown,17174.0,2437,3,No,11,"3,0",2,1,Unknown,3,3.000000,22,4,7,1975,yes


In [4]:
df_HR.shape

(1080, 32)

In [5]:
df_empleados = pd.read_csv('Employees.csv')
df_empleados.shape

(1080, 9)

df_employees a lista de tuplas

In [6]:
lista_de_tuplas = [tuple(x) for x in df_empleados.to_records(index=False)] 

In [11]:
lista_de_tuplas

[(51, 'No', 'Unknown', 6, 3, 'Unknown', 'M', 'Unknown', 1972),
 (52, 'No', 'Unknown', 1, 4, 'Life Sciences', 'M', 'Unknown', 1971),
 (42, 'No', 'travel_rarely', 4, 2, 'Technical Degree', 'M', 'Married', 1981),
 (47, 'No', 'travel_rarely', 2, 4, 'Medical', 'F', 'Married', 1976),
 (48, 'No', 'Unknown', 22, 3, 'Medical', 'F', 'Unknown', 1975),
 (59, 'No', 'Unknown', 25, 3, 'Life Sciences', 'F', 'Unknown', 1964),
 (42, 'No', 'travel_rarely', 1, 1, 'Unknown', 'M', 'Married', 1981),
 (41, 'No', 'travel_frequently', 9, 3, 'Unknown', 'M', 'Unknown', 1982),
 (41, 'No', 'non-travel', 7, 1, 'Unknown', 'F', 'Unknown', 1982),
 (41, 'Yes', 'travel_rarely', 12, 3, 'Unknown', 'F', 'Married', 1982),
 (40, 'No', 'travel_rarely', 2, 4, 'Unknown', 'F', 'Unknown', 1983),
 (58, 'Yes', 'travel_rarely', 2, 4, 'Unknown', 'M', 'Single', 1965),
 (45, 'No', 'non-travel', 1, 1, 'Unknown', 'M', 'Unknown', 1978),
 (33, 'No', 'Unknown', 1, 2, 'Life Sciences', 'F', 'Single', 1990),
 (47, 'No', 'Unknown', 2, 2, 'Other'

In [8]:
df_empleados.shape

(1080, 9)

Crear base de datos

In [9]:
cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
                              host='127.0.0.1')


mycursor = cnx.cursor()
try:
    mycursor.execute("CREATE DATABASE ABC_Corporation")
    print(mycursor)
except mysql.connector.Error as err:
    print(err)
    print("Error Code:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Message", err.msg)

CMySQLCursor: CREATE DATABASE ABC_Corporation


Creacion de tablas

In [10]:
cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
                              host='127.0.0.1', database='ABC_Corporation')


mycursor = cnx.cursor()
try:
    mycursor.execute("""
        CREATE TABLE Employees (
        EmployeeNumber INT PRIMARY KEY AUTO_INCREMENT,
        Age INT,
        Gender VARCHAR(100),
        MaritalStatus VARCHAR(100),
        DateBirth INT,
        Education INT,
        EducationField VARCHAR(100),
        Attrition VARCHAR(100),
        BusinessTravel VARCHAR(100),
        DistanceFromHome INT
    )
    """)
    print(mycursor)
except mysql.connector.Error as err:
    print(err)
    print("Error Code:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Message", err.msg)

CMySQLCursor: 
        CREATE TABLE Employees (
      ..


### Crear tabla JobDetails

In [17]:
cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
                              host='127.0.0.1', database='ABC_Corporation')


mycursor = cnx.cursor()
try:
    mycursor.execute("""
        CREATE TABLE JobDetails (
        JobDetailID INT PRIMARY KEY AUTO_INCREMENT,
        JobRole VARCHAR(100),
        JobLevel INT,
        NumCompaniesWorked INT,
        TotalWorkingYears VARCHAR(100),
        YearsAtCompany INT,
        YearsSinceLastPromotion INT,
        YearsWithCurrManager INT,
        EmployeeNumber INT,
        FOREIGN KEY (EmployeeNumber) REFERENCES Employees(EmployeeNumber)
    )
    """)
    print(mycursor)
except mysql.connector.Error as err:
    print(err)
    print("Error Code:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Message", err.msg)

1050 (42S01): Table 'jobdetails' already exists
Error Code: 1050
SQLSTATE 42S01
Message Table 'jobdetails' already exists


### Crear tabla Compensation

In [18]:
cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
                              host='127.0.0.1', database='ABC_Corporation')


mycursor = cnx.cursor()
try:
    mycursor.execute("""
        CREATE TABLE Compensation (
        CompensationID INT PRIMARY KEY AUTO_INCREMENT,
        EmployeeNumber INT,
        DailyRate INT,
        HourlyRate INT,
        MonthlyIncome INT,
        MonthlyRate INT,
        StockOptionLevel INT,
        PercentSalaryHike INT,
        OverTime VARCHAR(100),
        TrainingTimesLastYear INT,
        RemoteWork VARCHAR(100),
        FOREIGN KEY (EmployeeNumber) REFERENCES Employees(EmployeeNumber)
    )
    """)
    print(mycursor)
except mysql.connector.Error as err:
    print(err)
    print("Error Code:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Message", err.msg)

CMySQLCursor: 
        CREATE TABLE Compensation (
   ..


### Creacion tabla Satisfaction

In [19]:
cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
                              host='127.0.0.1', database='ABC_Corporation')


mycursor = cnx.cursor()
try:
    mycursor.execute("""
        CREATE TABLE Satisfaction (
        SatisfactionID INT PRIMARY KEY AUTO_INCREMENT,
        EmployeeNumber INT,
        EnvironmentSatisfaction INT,
        JobInvolvement INT,
        JobSatisfaction INT,
        RelationshipSatisfaction INT,
        WorkLifeBalance INT,
        PerformanceRating VARCHAR(100),
        FOREIGN KEY (EmployeeNumber) REFERENCES Employees(EmployeeNumber)
    )
    """)
    print(mycursor)
except mysql.connector.Error as err:
    print(err)
    print("Error Code:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Message", err.msg)

CMySQLCursor: 
        CREATE TABLE Satisfaction (
   ..
